# Extended Euclidean Algorithm 

This notebook demonstrates the Extended Euclidean Algorithm with verbose, step-by-step prints.
It is self-contained and intended to show how the multiplicative inverse of `b` modulo `m` is computed,
including the intermediate `q`, `t1/t2/t3` values and final verification.

The implementation is suitable for integers (used in classic modular arithmetic) and can be used to find inverses
needed for AES S-box construction (e.g. arithmetic in GF(2^8) using the AES modulus 0x11B).

## Contract
- Inputs: `m` (modulus), `b` (value to invert mod `m`). Both integers.
- Output: `b_inv` such that `(b * b_inv) % m == 1` if inverse exists; otherwise an exception is raised.
- Error modes: raises `ValueError` if no inverse exists (i.e. gcd(m,b) != 1).

We print each algorithm step to make the computation transparent.

In [1]:
def extended_euclidean_verbose(m, b):
    """Extended Euclidean Algorithm (verbose).
    Returns multiplicative inverse of b modulo m if it exists, printing full step trace.
    """
    a1, a2, a3 = 1, 0, m
    b1, b2, b3 = 0, 1, b
    q = 0
    # Header for the trace table
    print(f"{'q':>3} | {'a1':>6} | {'a2':>6} | {'a3':>10} | {'b1':>6} | {'b2':>6} | {'b3':>10}")
    print('-'*62)
    # Loop until remainder is 0 or 1
    while b3 != 0 and b3 != 1:
        print(f"{q:>3} | {a1:6} | {a2:6} | {a3:10} | {b1:6} | {b2:6} | {b3:10}")
        q = a3 // b3
        t1 = a1 - q * b1
        t2 = a2 - q * b2
        t3 = a3 - q * b3
        print(f"    q = {q}  ->  t1 = {t1}, t2 = {t2}, t3 = {t3}")
        # rotate rows (like the tabular extended-euclid method)
        a1, a2, a3 = b1, b2, b3
        b1, b2, b3 = t1, t2, t3
    # Final state
    print('-'*62)
    print(f"Final: a1={a1}, a2={a2}, a3={a3}  |  b1={b1}, b2={b2}, b3={b3}")
    if b3 == 0:
        # gcd != 1 -> no inverse
        raise ValueError(f"No multiplicative inverse for {b} mod {m} (GCD != 1)")
    # b3 == 1 -> b2 is the coefficient so that b*b2 + m*(something) = 1
    inverse = b2 % m
    print(f"Multiplicative inverse of {b} modulo {m} is: {inverse} (0x{inverse:02X})")
    return inverse


def validate_multiplicative_inverse(b, b_inv, m):
    product = (b * b_inv) % m
    print(f"Validation: ({b} * {b_inv}) % {m} = {product}")
    return product == 1


In [2]:
# Demonstrations: run the verbose algorithm on a few examples
# 1) Example from the repository: m = 1759, b = 550 (expected inverse 355)
m = 1759
b = 550
print("Example 1: m=1759, b=550")
inv1 = extended_euclidean_verbose(m, b)
print('Expected (from earlier notes): 355 ->', inv1, '\n')

# 2) AES-related small example: modulus m = 0x11B (AES polynomial as integer), try b = 0xC2
m = 0x11B  # AES irreducible polynomial represented as integer (decimal 283)
b = 0xC2
print("Example 2: AES field modulus m=0x11B (283), b=0xC2 (194)")
try:
    inv2 = extended_euclidean_verbose(m, b)
    ok = validate_multiplicative_inverse(b, inv2, m)
    print('Validation result:', ok)
except ValueError as e:
    print('Error:', e)

# 3) Small co-prime check: m = 26, b = 7 -> inverse should be 15 (because 7*15 = 105 = 1 mod 26)
print()
m = 26
b = 7
inv3 = extended_euclidean_verbose(m, b)
print('Check:', validate_multiplicative_inverse(b, inv3, m))


Example 1: m=1759, b=550
  q |     a1 |     a2 |         a3 |     b1 |     b2 |         b3
--------------------------------------------------------------
  0 |      1 |      0 |       1759 |      0 |      1 |        550
    q = 3  ->  t1 = 1, t2 = -3, t3 = 109
  3 |      0 |      1 |        550 |      1 |     -3 |        109
    q = 5  ->  t1 = -5, t2 = 16, t3 = 5
  5 |      1 |     -3 |        109 |     -5 |     16 |          5
    q = 21  ->  t1 = 106, t2 = -339, t3 = 4
 21 |     -5 |     16 |          5 |    106 |   -339 |          4
    q = 1  ->  t1 = -111, t2 = 355, t3 = 1
--------------------------------------------------------------
Final: a1=106, a2=-339, a3=4  |  b1=-111, b2=355, b3=1
Multiplicative inverse of 550 modulo 1759 is: 355 (0x163)
Expected (from earlier notes): 355 -> 355 

Example 2: AES field modulus m=0x11B (283), b=0xC2 (194)
  q |     a1 |     a2 |         a3 |     b1 |     b2 |         b3
--------------------------------------------------------------
  0 |   

# AES S-Box Construction

## Overview

The AES S-box (Substitution box) is a 16×16 lookup table used in the SubBytes transformation. It provides non-linearity in the cipher and is constructed through a mathematically rigorous process combining:
1. Multiplicative inverse in Galois Field GF(2⁸)
2. Affine transformation over GF(2)

## Construction Process

### Step 1: Initialize the S-box

Initialize a 16×16 table with byte values in ascending order (row-major):
```
     0   1   2   3   4   5   6   7   8   9   A   B   C   D   E   F
   ┌────────────────────────────────────────────────────────────────┐
 0 │ 00  01  02  03  04  05  06  07  08  09  0A  0B  0C  0D  0E  0F │
 1 │ 10  11  12  13  14  15  16  17  18  19  1A  1B  1C  1D  1E  1F │
 2 │ 20  21  22  23  24  25  26  27  28  29  2A  2B  2C  2D  2E  2F │
   │  ...                                                         ... │
 F │ F0  F1  F2  F3  F4  F5  F6  F7  F8  F9  FA  FB  FC  FD  FE  FF │
   └────────────────────────────────────────────────────────────────┘
```

- **Row 0**: {00}, {01}, {02}, ..., {0F}
- **Row 1**: {10}, {11}, {12}, ..., {1F}
- **Row F**: {F0}, {F1}, {F2}, ..., {FF}

### Step 2: Multiplicative Inverse in GF(2⁸)

Replace each byte with its multiplicative inverse in GF(2⁸) using the irreducible polynomial:

**m(x) = x⁸ + x⁴ + x³ + x + 1 (hex: 0x11B)**

For each byte **b** in the table:
- Find **b⁻¹** such that **b • b⁻¹ ≡ 1 (mod m(x))** in GF(2⁸)
- **Special case**: {00}⁻¹ = {00} (zero maps to itself)

**Example**:
```
Input: 0x53
Step 2: Find 0x53⁻¹ in GF(2⁸)
Result: 0x53⁻¹ = 0xCA
Verify: 0x53 • 0xCA = 0x01 in GF(2⁸) ✓
```

### Step 3: Affine Transformation

Apply an affine transformation over GF(2) to each byte from Step 2.

#### Transformation Formula

For each byte represented as 8 bits **(b₇, b₆, b₅, b₄, b₃, b₂, b₁, b₀)**:
```
b'ᵢ = bᵢ ⊕ b₍ᵢ₊₄₎ mod 8 ⊕ b₍ᵢ₊₅₎ mod 8 ⊕ b₍ᵢ₊₆₎ mod 8 ⊕ b₍ᵢ₊₇₎ mod 8 ⊕ cᵢ
```

where **i = 0, 1, 2, ..., 7** and ⊕ is XOR operation

#### Affine Constant

**c = {63} = (01100011)₂**

Binary representation: **(c₇, c₆, c₅, c₄, c₃, c₂, c₁, c₀) = (0, 1, 1, 0, 0, 0, 1, 1)**

#### Matrix Form

The affine transformation can also be expressed in matrix form:
```
┌───┐   ┌─────────────┐   ┌───┐   ┌───┐
│b'₀│   │1 0 0 0 1 1 1 1│   │b₀ │   │1 │
│b'₁│   │1 1 0 0 0 1 1 1│   │b₁ │   │1 │
│b'₂│   │1 1 1 0 0 0 1 1│   │b₂ │   │0 │
│b'₃│ = │1 1 1 1 0 0 0 1│ × │b₃ │ ⊕ │0 │
│b'₄│   │1 1 1 1 1 0 0 0│   │b₄ │   │0 │
│b'₅│   │0 1 1 1 1 1 0 0│   │b₅ │   │1 │
│b'₆│   │0 0 1 1 1 1 1 0│   │b₆ │   │1 │
│b'₇│   │0 0 0 1 1 1 1 1│   │b₇ │   │0 │
└───┘   └─────────────┘   └───┘   └───┘
```

### Complete Example: S-box[0x53] = 0xED
```
Input: 0x53 = 01010011₂

Step 1: Initial value
  0x53

Step 2: Multiplicative inverse in GF(2⁸)
  0x53⁻¹ = 0xCA = 11001010₂
  
Step 3: Affine transformation on 0xCA
  Input bits:  b₇ b₆ b₅ b₄ b₃ b₂ b₁ b₀
               1  1  0  0  1  0  1  0
  
  Bit-by-bit calculation:
  b'₀ = b₀ ⊕ b₄ ⊕ b₅ ⊕ b₆ ⊕ b₇ ⊕ c₀ = 0⊕1⊕0⊕1⊕1⊕1 = 0
  b'₁ = b₁ ⊕ b₅ ⊕ b₆ ⊕ b₇ ⊕ b₀ ⊕ c₁ = 1⊕0⊕1⊕1⊕0⊕1 = 0
  b'₂ = b₂ ⊕ b₆ ⊕ b₇ ⊕ b₀ ⊕ b₁ ⊕ c₂ = 0⊕1⊕1⊕0⊕1⊕0 = 1
  b'₃ = b₃ ⊕ b₇ ⊕ b₀ ⊕ b₁ ⊕ b₂ ⊕ c₃ = 1⊕1⊕0⊕1⊕0⊕0 = 1
  b'₄ = b₄ ⊕ b₀ ⊕ b₁ ⊕ b₂ ⊕ b₃ ⊕ c₄ = 1⊕0⊕1⊕0⊕1⊕0 = 1
  b'₅ = b₅ ⊕ b₁ ⊕ b₂ ⊕ b₃ ⊕ b₄ ⊕ c₅ = 0⊕1⊕0⊕1⊕1⊕1 = 0
  b'₆ = b₆ ⊕ b₂ ⊕ b₃ ⊕ b₄ ⊕ b₅ ⊕ c₆ = 1⊕0⊕1⊕1⊕0⊕1 = 1
  b'₇ = b₇ ⊕ b₃ ⊕ b₄ ⊕ b₅ ⊕ b₆ ⊕ c₇ = 1⊕1⊕1⊕0⊕1⊕0 = 1
  
  Output bits: 1  1  0  1  1  1  0  0
               b₇ b₆ b₅ b₄ b₃ b₂ b₁ b₀
  
Final Result: 0xED = 11101101₂

✓ Verification: S-box[0x53] = 0xED
```

## Final AES S-Box (partial)
```
     0   1   2   3   4   5   6   7   8   9   A   B   C   D   E   F
   ┌────────────────────────────────────────────────────────────────┐
 0 │ 63  7C  77  7B  F2  6B  6F  C5  30  01  67  2B  FE  D7  AB  76 │
 1 │ CA  82  C9  7D  FA  59  47  F0  AD  D4  A2  AF  9C  A4  72  C0 │
 2 │ B7  FD  93  26  36  3F  F7  CC  34  A5  E5  F1  71  D8  31  15 │
 3 │ 04  C7  23  C3  18  96  05  9A  07  12  80  E2  EB  27  B2  75 │
   │                          ...                                    │
 5 │ 53  D1  00  ED  20  FC  B1  5B  6A  CB  BE  39  4A  4C  58  CF │
   │                          ...                                    │
   └────────────────────────────────────────────────────────────────┘
```

## Inverse S-Box

The inverse S-box is constructed by reversing the process:
1. Apply **inverse affine transformation** (different matrix and constant {05})
2. Take **multiplicative inverse** in GF(2⁸)

This ensures: **InvS-box(S-box(x)) = x** for all bytes x

## Key Properties

1. **Non-linearity**: No byte in the S-box can be expressed as a linear combination of others
2. **Algebraic complexity**: High resistance to algebraic attacks
3. **Invertibility**: Each byte maps to a unique byte (bijection)
4. **Avalanche effect**: Small input changes cause large output changes
5. **No fixed points**: S-box(x) ≠ x for all x (except construction process)

## Mathematical Justification

- **Multiplicative inverse**: Provides non-linearity and algebraic complexity
- **Affine transformation**: Eliminates fixed points and adds diffusion
- **Constant {63}**: Chosen to maximize distance from any fixed point

In [3]:
# Build AES S-box (multiplicative inverse in GF(2^8) + AES affine transform)

def gf_mul(a, b):
    """Galois Field (2^8) multiplication used by AES (irreducible poly x^8 + x^4 + x^3 + x + 1).
    This is the usual bytewise implementation using 0x1B reduction on carry.
    """
    p = 0
    for _ in range(8):
        if b & 1:
            p ^= a
        carry = a & 0x80
        a = (a << 1) & 0xFF
        if carry:
            a ^= 0x1B
        b >>= 1
    return p & 0xFF


def gf_pow(a, power):
    """Exponentiation in GF(2^8) using repeated squaring.
    For multiplicative inverse use power = 254 (since a^(2^8-2) = a^-1 for a != 0).
    """
    result = 1
    base = a
    while power > 0:
        if power & 1:
            result = gf_mul(result, base)
        base = gf_mul(base, base)
        power >>= 1
    return result & 0xFF


def affine_transform(byte):
    """AES affine transform applied to a byte (after multiplicative inverse).
    Uses the standard bitwise definition with constant 0x63.
    """
    c = 0x63
    out = 0
    for i in range(8):
        # XOR of the five bits: b_i ^ b_{i+4} ^ b_{i+5} ^ b_{i+6} ^ b_{i+7} (indices mod 8)
        print(f"Calculating bit {i} of affine transform")
        bit = ((byte >> i) & 1) ^ ((byte >> ((i + 4) % 8)) & 1) ^ ((byte >> ((i + 5) % 8)) & 1) ^ ((byte >> ((i + 6) % 8)) & 1) ^ ((byte >> ((i + 7) % 8)) & 1) ^ ((c >> i) & 1)
        out |= (bit << i)
    return out & 0xFF


def affine_transform_verbose(byte):
    """
    Verbose AES affine transform with detailed step-by-step output.
    Shows formula substitution, binary and hex representations.
    """
    c = 0x63
    
    print(f"\n{'='*80}")
    print(f"AFFINE TRANSFORMATION")
    print(f"{'='*80}")
    print(f"\nInput byte:  0x{byte:02X} = {byte:08b}b = {byte:3d} (decimal)")
    print(f"Constant c:  0x{c:02X} = {c:08b}b = {c:3d} (decimal)")
    print(f"             bit: 76543210")
    
    print(f"\n{'─'*80}")
    print(f"FORMULA: b'ᵢ = bᵢ ⊕ b₍ᵢ₊₄₎ ⊕ b₍ᵢ₊₅₎ ⊕ b₍ᵢ₊₆₎ ⊕ b₍ᵢ₊₇₎ ⊕ cᵢ (all indices mod 8)")
    print(f"{'─'*80}")
    
    # Extract all input bits for easy reference
    input_bits = [(byte >> i) & 1 for i in range(8)]
    c_bits = [(c >> i) & 1 for i in range(8)]
    
    print(f"\nInput bit values:")
    print(f"  b₇ b₆ b₅ b₄ b₃ b₂ b₁ b₀")
    print(f"  {' '.join(str(b) for b in reversed(input_bits))}")
    
    print(f"\nConstant c bit values:")
    print(f"  c₇ c₆ c₅ c₄ c₃ c₂ c₁ c₀")
    print(f"  {' '.join(str(b) for b in reversed(c_bits))}")
    
    out = 0
    output_bits = []
    
    print(f"\n{'='*80}")
    print(f"BIT-BY-BIT CALCULATIONS")
    print(f"{'='*80}")
    
    for i in range(8):
        print(f"\n┌{'─'*76}┐")
        print(f"│ BIT {i} CALCULATION{' '*60}│")
        print(f"└{'─'*76}┘")
        
        # Calculate indices (mod 8)
        idx_0 = i
        idx_4 = (i + 4) % 8
        idx_5 = (i + 5) % 8
        idx_6 = (i + 6) % 8
        idx_7 = (i + 7) % 8
        
        # Get bit values
        b_i = (byte >> idx_0) & 1
        b_i_plus_4 = (byte >> idx_4) & 1
        b_i_plus_5 = (byte >> idx_5) & 1
        b_i_plus_6 = (byte >> idx_6) & 1
        b_i_plus_7 = (byte >> idx_7) & 1
        c_i = (c >> i) & 1
        
        # Calculate result bit
        result_bit = b_i ^ b_i_plus_4 ^ b_i_plus_5 ^ b_i_plus_6 ^ b_i_plus_7 ^ c_i
        
        # Formula with substituted indices
        print(f"\nFormula with indices:")
        print(f"  b'_{i} = b_{idx_0} ⊕ b_{idx_4} ⊕ b_{idx_5} ⊕ b_{idx_6} ⊕ b_{idx_7} ⊕ c_{i}")
        
        # Show bit extraction
        print(f"\nBit extraction from input byte 0x{byte:02X} = {byte:08b}b:")
        print(f"  b_{idx_0} = bit {idx_0} = {b_i}  (extracted: (0x{byte:02X} >> {idx_0}) & 1)")
        print(f"  b_{idx_4} = bit {idx_4} = {b_i_plus_4}  (extracted: (0x{byte:02X} >> {idx_4}) & 1)")
        print(f"  b_{idx_5} = bit {idx_5} = {b_i_plus_5}  (extracted: (0x{byte:02X} >> {idx_5}) & 1)")
        print(f"  b_{idx_6} = bit {idx_6} = {b_i_plus_6}  (extracted: (0x{byte:02X} >> {idx_6}) & 1)")
        print(f"  b_{idx_7} = bit {idx_7} = {b_i_plus_7}  (extracted: (0x{byte:02X} >> {idx_7}) & 1)")
        print(f"  c_{i}  = bit {i} = {c_i}  (extracted: (0x{c:02X} >> {i}) & 1)")
        
        # Show substituted formula
        print(f"\nSubstituted formula:")
        print(f"  b'_{i} = {b_i} ⊕ {b_i_plus_4} ⊕ {b_i_plus_5} ⊕ {b_i_plus_6} ⊕ {b_i_plus_7} ⊕ {c_i}")
        
        # Show XOR chain step by step
        print(f"\nXOR chain calculation:")
        step1 = b_i ^ b_i_plus_4
        print(f"  Step 1: {b_i} ⊕ {b_i_plus_4} = {step1}")
        
        step2 = step1 ^ b_i_plus_5
        print(f"  Step 2: {step1} ⊕ {b_i_plus_5} = {step2}")
        
        step3 = step2 ^ b_i_plus_6
        print(f"  Step 3: {step2} ⊕ {b_i_plus_6} = {step3}")
        
        step4 = step3 ^ b_i_plus_7
        print(f"  Step 4: {step3} ⊕ {b_i_plus_7} = {step4}")
        
        step5 = step4 ^ c_i
        print(f"  Step 5: {step4} ⊕ {c_i} = {step5}")
        
        print(f"\n✓ Result: b'_{i} = {result_bit}")
        
        # Store bit
        out |= (result_bit << i)
        output_bits.append(result_bit)
    
    # Summary table
    print(f"\n{'='*80}")
    print(f"SUMMARY TABLE")
    print(f"{'='*80}")
    print(f"\n┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬────────┬────────────────────────┐")
    print(f"│ bit │  bᵢ │b₍ᵢ₊₄₎│b₍ᵢ₊₅₎│b₍ᵢ₊₆₎│b₍ᵢ₊₇₎│  cᵢ │  b'ᵢ   │ Formula Result         │")
    print(f"├─────┼─────┼─────┼─────┼─────┼─────┼─────┼────────┼────────────────────────┤")
    
    for i in range(8):
        idx_4 = (i + 4) % 8
        idx_5 = (i + 5) % 8
        idx_6 = (i + 6) % 8
        idx_7 = (i + 7) % 8
        
        b_i = (byte >> i) & 1
        b_i_plus_4 = (byte >> idx_4) & 1
        b_i_plus_5 = (byte >> idx_5) & 1
        b_i_plus_6 = (byte >> idx_6) & 1
        b_i_plus_7 = (byte >> idx_7) & 1
        c_i = (c >> i) & 1
        result_bit = output_bits[i]
        
        formula = f"{b_i}⊕{b_i_plus_4}⊕{b_i_plus_5}⊕{b_i_plus_6}⊕{b_i_plus_7}⊕{c_i}={result_bit}"
        
        print(f"│  {i}  │  {b_i}  │  {b_i_plus_4}  │  {b_i_plus_5}  │  {b_i_plus_6}  │  {b_i_plus_7}  │  {c_i}  │   {result_bit}    │ {formula:22} │")
    
    print(f"└─────┴─────┴─────┴─────┴─────┴─────┴─────┴────────┴────────────────────────┘")
    
    # Final result
    print(f"\n{'='*80}")
    print(f"FINAL RESULT")
    print(f"{'='*80}")
    
    print(f"\nOutput bits (MSB to LSB):")
    print(f"  b'₇ b'₆ b'₅ b'₄ b'₃ b'₂ b'₁ b'₀")
    print(f"  {' '.join(str(b) for b in reversed(output_bits))}")
    
    print(f"\nOutput byte:")
    print(f"  Binary:  {out:08b}b")
    print(f"  Hex:     0x{out:02X}")
    print(f"  Decimal: {out}")
    
    print(f"\n{'─'*80}")
    print(f"TRANSFORMATION COMPLETE")
    print(f"  Input:  0x{byte:02X} = {byte:08b}b")
    print(f"  Output: 0x{out:02X} = {out:08b}b")
    print(f"{'─'*80}\n")
    
    return out & 0xFF


# Generate S-box
sbox_generated = [0] * 256
for x in range(256):
    if x == 0:
        inv = 0
    else:
        inv = gf_pow(x, 254)  # multiplicative inverse in GF(2^8)
    sbox_generated[x] = affine_transform(inv)

# Reference AES S-box (standard)
AES_SBOX = [
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
]

# Compare
match = sbox_generated == AES_SBOX
print('S-box match with reference:', match)
if not match:
    print('\nMismatches (index, generated, reference):')
    for i, (g, r) in enumerate(zip(sbox_generated, AES_SBOX)):
        if g != r:
            print(f'{i:02X}: 0x{g:02X} != 0x{r:02X}')
else:
    # print compact summary of first 16 bytes
    print('\nFirst 16 bytes of generated S-box:')
    print(' '.join(f"{x:02X}" for x in sbox_generated[:16]))

# Build inverse S-box and sanity-check
inv_sbox = [0]*256
for i, val in enumerate(sbox_generated):
    inv_sbox[val] = i
# Validate by applying sbox then inv_sbox maps back a few sample values
samples = [0x00, 0x53, 0xA7, 0xFF]
print('\nRound-trip checks:')
for s in samples:
    r = inv_sbox[sbox_generated[s]]
    print(f'0x{s:02X} -> Sbox 0x{sbox_generated[s]:02X} -> Inv 0x{r:02X}')


Calculating bit 0 of affine transform
Calculating bit 1 of affine transform
Calculating bit 2 of affine transform
Calculating bit 3 of affine transform
Calculating bit 4 of affine transform
Calculating bit 5 of affine transform
Calculating bit 6 of affine transform
Calculating bit 7 of affine transform
Calculating bit 0 of affine transform
Calculating bit 1 of affine transform
Calculating bit 2 of affine transform
Calculating bit 3 of affine transform
Calculating bit 4 of affine transform
Calculating bit 5 of affine transform
Calculating bit 6 of affine transform
Calculating bit 7 of affine transform
Calculating bit 0 of affine transform
Calculating bit 1 of affine transform
Calculating bit 2 of affine transform
Calculating bit 3 of affine transform
Calculating bit 4 of affine transform
Calculating bit 5 of affine transform
Calculating bit 6 of affine transform
Calculating bit 7 of affine transform
Calculating bit 0 of affine transform
Calculating bit 1 of affine transform
Calculating 

In [5]:
def verify_sbox_data(hex_input, hex_expected):
    # Verification by computing affine transformation from scratch
    # (using multiplicative inverse in GF(2^8) + affine transform)
    if hex_input == 0:
        inv = 0
    else:
        inv = gf_pow(hex_input, 254)  # multiplicative inverse in GF(2^8)
    
    print(f"Computing S-box for input 0x{hex_input:02X}:")
    print(f"  Multiplicative inverse in GF(2^8) of 0x{hex_input:02X}: 0x{inv:02X}")

    transformed = affine_transform_verbose(inv)
    print(f"Input: 0x{hex_input:02X}, Expected S-box: 0x{hex_expected:02X}, Computed S-box: 0x{transformed:02X}")
    assert transformed == hex_expected, "S-box computation mismatch!" 

#verify_sbox_data(0x0c, 0xfe)

#  AES implementation

In [4]:
# AES core step implementations with enhanced visual logging

def print_state_boxed(state, title=None):
    """Print state in a nice boxed format"""
    if title:
        print(f"\n{title}")
    print("┌─────────────────┐")
    for r in range(4):
        print("│ " + '  '.join(f"{state[r][c]:02X}" for c in range(4)) + " │")
    print("└─────────────────┘")


def print_side_by_side_states(state1, state2, title1="Before", title2="After", operation=""):
    """Print two states side by side for comparison"""
    if operation:
        print(f"\n{'='*60}")
        print(f"{operation}")
        print('='*60)
    
    print(f"\n{title1:^22}      {title2:^22}")
    print("┌─────────────────┐      ┌─────────────────┐")
    for r in range(4):
        left = '  '.join(f"{state1[r][c]:02X}" for c in range(4))
        right = '  '.join(f"{state2[r][c]:02X}" for c in range(4))
        print(f"│ {left} │  =>  │ {right} │")
    print("└─────────────────┘      └─────────────────┘")


def sub_bytes(state, sbox):
    """Apply S-box substitution to every byte of the state."""
    result = [[sbox[state[r][c]] for c in range(4)] for r in range(4)]
    
    print(f"\n{'='*60}")
    print("SUBBYTES OPERATION")
    print('='*60)
    print("\nS-box substitution: Each byte is replaced using the S-box lookup")
    
    # Print transformation in matrix format
    print(f"\n{'INPUT STATE':^22}      {'OUTPUT STATE':^22}")
    print("┌─────────────────┐      ┌─────────────────┐")
    for r in range(4):
        left = '  '.join(f"{state[r][c]:02X}" for c in range(4))
        right = '  '.join(f"{result[r][c]:02X}" for c in range(4))
        print(f"│ {left} │  =>  │ {right} │")
    print("└─────────────────┘      └─────────────────┘")
    
    # Print detailed substitution table
    print("\nDetailed S-box substitutions:")
    print("┌─────┬─────┬──────────┬──────────┐")
    print("│ Row │ Col │  Input   │  Output  │")
    print("├─────┼─────┼──────────┼──────────┤")
    for r in range(4):
        for c in range(4):
            s_byte = state[r][c]
            sub_byte = sbox[s_byte]
            print(f"│  {r}  │  {c}  │   0x{s_byte:02X}   │   0x{sub_byte:02X}   │")
    print("└─────┴─────┴──────────┴──────────┘")
    
    return result


def inv_sub_bytes(state, inv_sbox):
    """Apply inverse S-box substitution to every byte of the state."""
    result = [[inv_sbox[state[r][c]] for c in range(4)] for r in range(4)]
    
    print(f"\n{'='*60}")
    print("INVERSE SUBBYTES OPERATION")
    print('='*60)
    
    print_side_by_side_states(state, result, "INPUT STATE", "OUTPUT STATE", "")
    
    return result


def shift_rows(state):
    """Shift rows to the left by row index (row 0 no shift)."""
    out = [list(row) for row in state]
    for r in range(1, 4):
        out[r] = state[r][r:] + state[r][:r]
    
    print(f"\n{'='*60}")
    print("SHIFTROWS OPERATION")
    print('='*60)
    print("\nRow 0: No shift")
    print("Row 1: Shift left by 1")
    print("Row 2: Shift left by 2")
    print("Row 3: Shift left by 3")
    
    # Visual representation with arrows
    print(f"\n{'INPUT STATE':^22}      {'OUTPUT STATE':^22}")
    print("┌─────────────────┐      ┌─────────────────┐")
    
    for r in range(4):
        left = '  '.join(f"{state[r][c]:02X}" for c in range(4))
        right = '  '.join(f"{out[r][c]:02X}" for c in range(4))
        shift_info = f"← {r}" if r > 0 else "   "
        print(f"│ {left} │ {shift_info} │ {right} │")
    print("└─────────────────┘      └─────────────────┘")
    
    # Show detailed row-by-row shifts
    print("\nDetailed row shifts:")
    for r in range(4):
        if r == 0:
            print(f"Row {r}: [{' '.join(f'{state[r][c]:02X}' for c in range(4))}] (no shift)")
        else:
            before = ' '.join(f'{state[r][c]:02X}' for c in range(4))
            after = ' '.join(f'{out[r][c]:02X}' for c in range(4))
            print(f"Row {r}: [{before}] → [{after}] (shift left {r})")
    
    return out


def inv_shift_rows(state):
    """Shift rows to the right (inverse operation)."""
    out = [list(row) for row in state]
    for r in range(1, 4):
        out[r] = state[r][-r:] + state[r][:-r]
    
    print(f"\n{'='*60}")
    print("INVERSE SHIFTROWS OPERATION")
    print('='*60)
    print("\nRow 0: No shift")
    print("Row 1: Shift right by 1")
    print("Row 2: Shift right by 2")
    print("Row 3: Shift right by 3")
    
    print_side_by_side_states(state, out, "INPUT STATE", "OUTPUT STATE", "")
    
    return out


def mix_single_column(col, verbose=False, col_index=0):
    """Mix one column (4 bytes) using AES forward matrix."""
    s0, s1, s2, s3 = col
    
    if verbose:
        print(f"\n  Column {col_index}: [{' '.join(f'0x{b:02X}' for b in col)}]")
        print(f"  ┌────────────────────────────────────────────────┐")
        print(f"  │ Matrix multiplication in GF(2^8):              │")
        print(f"  │ ┌──────────────┐   ┌────┐   ┌────┐            │")
        print(f"  │ │ 02 03 01 01  │   │ {s0:02X} │   │ r0 │            │")
        print(f"  │ │ 01 02 03 01  │ × │ {s1:02X} │ = │ r1 │            │")
        print(f"  │ │ 01 01 02 03  │   │ {s2:02X} │   │ r2 │            │")
        print(f"  │ │ 03 01 01 02  │   │ {s3:02X} │   │ r3 │            │")
        print(f"  │ └──────────────┘   └────┘   └────┘            │")
        print(f"  └────────────────────────────────────────────────┘")
    
    r0 = (gf_mul(0x02, s0) ^ gf_mul(0x03, s1) ^ s2 ^ s3) & 0xFF
    r1 = (s0 ^ gf_mul(0x02, s1) ^ gf_mul(0x03, s2) ^ s3) & 0xFF
    r2 = (s0 ^ s1 ^ gf_mul(0x02, s2) ^ gf_mul(0x03, s3)) & 0xFF
    r3 = (gf_mul(0x03, s0) ^ s1 ^ s2 ^ gf_mul(0x02, s3)) & 0xFF
    
    if verbose:
        print(f"  Calculations:")
        print(f"  r0 = (02•{s0:02X}) ⊕ (03•{s1:02X}) ⊕ {s2:02X} ⊕ {s3:02X} = 0x{r0:02X}")
        print(f"  r1 = {s0:02X} ⊕ (02•{s1:02X}) ⊕ (03•{s2:02X}) ⊕ {s3:02X} = 0x{r1:02X}")
        print(f"  r2 = {s0:02X} ⊕ {s1:02X} ⊕ (02•{s2:02X}) ⊕ (03•{s3:02X}) = 0x{r2:02X}")
        print(f"  r3 = (03•{s0:02X}) ⊕ {s1:02X} ⊕ {s2:02X} ⊕ (02•{s3:02X}) = 0x{r3:02X}")
        print(f"  Result: [0x{r0:02X}, 0x{r1:02X}, 0x{r2:02X}, 0x{r3:02X}]")
    
    return [r0, r1, r2, r3]


def mix_columns(state):
    """Apply MixColumns to the state."""
    print(f"\n{'='*60}")
    print("MIXCOLUMNS OPERATION")
    print('='*60)
    print("\nEach column is multiplied by the fixed matrix:")
    print("┌──────────────┐")
    print("│ 02 03 01 01  │")
    print("│ 01 02 03 01  │")
    print("│ 01 01 02 03  │")
    print("│ 03 01 01 02  │")
    print("└──────────────┘")
    print("(All operations in GF(2^8) with modulo x^8 + x^4 + x^3 + x + 1)")
    
    out = [[0]*4 for _ in range(4)]
    
    for c in range(4):
        col = [state[r][c] for r in range(4)]
        mixed = mix_single_column(col, verbose=True, col_index=c)
        for r in range(4):
            out[r][c] = mixed[r]
    
    # Show before/after
    print(f"\n{'INPUT STATE':^22}      {'OUTPUT STATE':^22}")
    print("┌─────────────────┐      ┌─────────────────┐")
    for r in range(4):
        left = '  '.join(f"{state[r][c]:02X}" for c in range(4))
        right = '  '.join(f"{out[r][c]:02X}" for c in range(4))
        print(f"│ {left} │  =>  │ {right} │")
    print("└─────────────────┘      └─────────────────┘")
    
    return out


def inv_mix_single_column(col):
    """Inverse mix column operation."""
    s0, s1, s2, s3 = col
    r0 = (gf_mul(0x0E, s0) ^ gf_mul(0x0B, s1) ^ gf_mul(0x0D, s2) ^ gf_mul(0x09, s3)) & 0xFF
    r1 = (gf_mul(0x09, s0) ^ gf_mul(0x0E, s1) ^ gf_mul(0x0B, s2) ^ gf_mul(0x0D, s3)) & 0xFF
    r2 = (gf_mul(0x0D, s0) ^ gf_mul(0x09, s1) ^ gf_mul(0x0E, s2) ^ gf_mul(0x0B, s3)) & 0xFF
    r3 = (gf_mul(0x0B, s0) ^ gf_mul(0x0D, s1) ^ gf_mul(0x09, s2) ^ gf_mul(0x0E, s3)) & 0xFF
    return [r0, r1, r2, r3]


def inv_mix_columns(state):
    """Apply inverse MixColumns to the state."""
    print(f"\n{'='*60}")
    print("INVERSE MIXCOLUMNS OPERATION")
    print('='*60)
    
    out = [[0]*4 for _ in range(4)]
    for c in range(4):
        col = [state[r][c] for r in range(4)]
        mixed = inv_mix_single_column(col)
        for r in range(4):
            out[r][c] = mixed[r]
    
    print_side_by_side_states(state, out, "INPUT STATE", "OUTPUT STATE", "")
    
    return out


def add_round_key(state, round_key):
    """XOR the state with the round key (both 4x4)."""
    result = [[state[r][c] ^ round_key[r][c] for c in range(4)] for r in range(4)]
    
    print(f"\n{'='*60}")
    print("ADDROUNDKEY OPERATION")
    print('='*60)
    print("\nXOR state with round key (⊕ operation)")
    
    # Three-way display: State ⊕ Key = Result
    print(f"\n{'STATE':^19}   {'ROUND KEY':^19}   {'RESULT':^19}")
    print("┌─────────────────┐   ┌─────────────────┐   ┌─────────────────┐")
    for r in range(4):
        state_row = '  '.join(f"{state[r][c]:02X}" for c in range(4))
        key_row = '  '.join(f"{round_key[r][c]:02X}" for c in range(4))
        result_row = '  '.join(f"{result[r][c]:02X}" for c in range(4))
        print(f"│ {state_row} │ ⊕ │ {key_row} │ = │ {result_row} │")
    print("└─────────────────┘   └─────────────────┘   └─────────────────┘")
    
    # Detailed XOR table
    print("\nDetailed XOR operations:")
    print("┌─────┬─────┬────────┬────────┬────────┐")
    print("│ Row │ Col │  State │   Key  │ Result │")
    print("├─────┼─────┼────────┼────────┼────────┤")
    for r in range(4):
        for c in range(4):
            s_byte = state[r][c]
            k_byte = round_key[r][c]
            res_byte = result[r][c]
            print(f"│  {r}  │  {c}  │  0x{s_byte:02X}  │  0x{k_byte:02X}  │  0x{res_byte:02X}  │")
    print("└─────┴─────┴────────┴────────┴────────┘")
    
    return result


# Initialize S-boxes
try:
    sbox_ref = AES_SBOX
except NameError:
    sbox_ref = sbox_generated

inv_sbox_ref = [0]*256
for i, v in enumerate(sbox_ref):
    inv_sbox_ref[v] = i

# Print S-boxes in a nice format
print("\n" + "="*60)
print("AES S-BOX")
print("="*60)
print("     0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F")
print("   ┌────────────────────────────────────────────────┐")
for i in range(16):
    row_hex = f"{i:X}"
    values = ' '.join(f"{sbox_ref[i*16 + j]:02X}" for j in range(16))
    print(f" {row_hex} │ {values} │")
print("   └────────────────────────────────────────────────┘")

print("\n" + "="*60)
print("AES INVERSE S-BOX")
print("="*60)
print("     0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F")
print("   ┌────────────────────────────────────────────────┐")
for i in range(16):
    row_hex = f"{i:X}"
    values = ' '.join(f"{inv_sbox_ref[i*16 + j]:02X}" for j in range(16))
    print(f" {row_hex} │ {values} │")
print("   └────────────────────────────────────────────────┘")

# Example demonstration
print("\n" + "="*70)
print("AES TRANSFORMATION DEMONSTRATION")
print("="*70)

sample_state = [
    [0x32, 0x88, 0x31, 0xE0],
    [0x43, 0x5A, 0x31, 0x37],
    [0xF6, 0x30, 0x98, 0x07],
    [0xA8, 0x8D, 0xA2, 0x34]
]

print_state_boxed(sample_state, '\nInitial State:')

# SubBytes
s1 = sub_bytes(sample_state, sbox_ref)

# ShiftRows
s2 = shift_rows(s1)

# MixColumns
s3 = mix_columns(s2)

# AddRoundKey (use a sample key)
sample_key = [
    [0x2B, 0x28, 0xAB, 0x09],
    [0x7E, 0xAE, 0xF7, 0xCF],
    [0x15, 0xD2, 0x15, 0x4F],
    [0x16, 0xA6, 0x88, 0x3C]
]
s4 = add_round_key(s3, sample_key)

# Demonstrate inverse operations
print("\n" + "="*70)
print("VERIFYING INVERSE OPERATIONS")
print("="*70)

#s_inv_mix = inv_mix_columns(s3)
#print(f"\n✓ InvMixColumns matches ShiftRows result: {s_inv_mix == s2}")
#
#s_inv_shift = inv_shift_rows(s2)
#print(f"✓ InvShiftRows matches SubBytes result: {s_inv_shift == s1}")
#
#s_inv_sub = inv_sub_bytes(s1, inv_sbox_ref)
#print(f"✓ InvSubBytes matches initial state: {s_inv_sub == sample_state}")

NameError: name 'sbox_generated' is not defined

In [3]:
def gf_mul_verbose(a, b):
    """Verbose Galois Field multiplication showing all binary operations.
    Only works for b = 0x02 or b = 0x03 (common in AES).
    """
    if b == 0x02:
        print(f"      Multiplying 0x{a:02X} by 0x02 (left shift with conditional XOR):")
        print(f"        Input:       0x{a:02X} = {a:08b}b")
        
        # Check if high bit is set
        high_bit_set = (a & 0x80) != 0
        print(f"        High bit 7:  {1 if high_bit_set else 0} (bit 7 = {(a >> 7) & 1})")
        
        # Left shift
        shifted = (a << 1) & 0xFF
        print(f"        Left shift:  0x{a:02X} << 1 = 0x{shifted:02X} = {shifted:08b}b")
        
        # Conditional XOR with 0x1B
        if high_bit_set:
            result = shifted ^ 0x1B
            print(f"        High bit was 1, so XOR with 0x1B (irreducible polynomial):")
            print(f"          {shifted:08b}b  (shifted)")
            print(f"        ⊕ {0x1B:08b}b  (0x1B)")
            print(f"        = {result:08b}b  (0x{result:02X})")
        else:
            result = shifted
            print(f"        High bit was 0, no XOR needed")
            print(f"        Result: 0x{result:02X}")
        
        return result
    
    elif b == 0x03:
        print(f"      Multiplying 0x{a:02X} by 0x03:")
        print(f"        0x03 = 0x02 ⊕ 0x01, so (0x03 • a) = (0x02 • a) ⊕ a")
        print(f"        Step 1: Calculate (0x02 • 0x{a:02X})")
        
        # First multiply by 0x02
        mul2 = gf_mul_verbose(a, 0x02)
        
        # Then XOR with original
        result = mul2 ^ a
        print(f"        Step 2: XOR result with original value:")
        print(f"          {mul2:08b}b  (0x02 • 0x{a:02X} = 0x{mul2:02X})")
        print(f"        ⊕ {a:08b}b  (original 0x{a:02X})")
        print(f"        = {result:08b}b  (0x{result:02X})")
        
        return result
    
    else:
        # Fallback to regular gf_mul for other values
        return gf_mul(a, b)


def mix_single_column_verbose(col, col_index=0):
    """Mix one column with explicit binary GF(2^8) operations."""
    s0, s1, s2, s3 = col
    
    print(f"\n  {'='*70}")
    print(f"  Column {col_index}: [0x{s0:02X}, 0x{s1:02X}, 0x{s2:02X}, 0x{s3:02X}]")
    print(f"  {'='*70}")
    print(f"  Matrix multiplication in GF(2^8):")
    print(f"  ┌──────────────┐   ┌────┐   ┌────┐")
    print(f"  │ 02 03 01 01  │   │ {s0:02X} │   │ r0 │")
    print(f"  │ 01 02 03 01  │ × │ {s1:02X} │ = │ r1 │")
    print(f"  │ 01 01 02 03  │   │ {s2:02X} │   │ r2 │")
    print(f"  │ 03 01 01 02  │   │ {s3:02X} │   │ r3 │")
    print(f"  └──────────────┘   └────┘   └────┘")
    print()
    
    # Calculate r0 = (02•s0) ⊕ (03•s1) ⊕ s2 ⊕ s3
    print(f"  ─── Calculating r0 = (0x02 • 0x{s0:02X}) ⊕ (0x03 • 0x{s1:02X}) ⊕ 0x{s2:02X} ⊕ 0x{s3:02X} ───")
    print()
    
    print(f"    Term 1: (0x02 • 0x{s0:02X})")
    term1 = gf_mul_verbose(s0, 0x02)
    print()
    
    print(f"    Term 2: (0x03 • 0x{s1:02X})")
    term2 = gf_mul_verbose(s1, 0x03)
    print()
    
    print(f"    Term 3: 0x{s2:02X} (identity, no multiplication)")
    term3 = s2
    print(f"      Value: {term3:08b}b = 0x{term3:02X}")
    print()
    
    print(f"    Term 4: 0x{s3:02X} (identity, no multiplication)")
    term4 = s3
    print(f"      Value: {term4:08b}b = 0x{term4:02X}")
    print()
    
    print(f"    Final XOR of all terms:")
    print(f"      {term1:08b}b  (0x{term1:02X}) ← Term 1")
    print(f"    ⊕ {term2:08b}b  (0x{term2:02X}) ← Term 2")
    print(f"    ⊕ {term3:08b}b  (0x{term3:02X}) ← Term 3")
    print(f"    ⊕ {term4:08b}b  (0x{term4:02X}) ← Term 4")
    
    r0 = term1 ^ term2 ^ term3 ^ term4
    print(f"    = {r0:08b}b  (0x{r0:02X}) ← r0")
    print()
    
    # Calculate r1 = s0 ⊕ (02•s1) ⊕ (03•s2) ⊕ s3
    print(f"  ─── Calculating r1 = 0x{s0:02X} ⊕ (0x02 • 0x{s1:02X}) ⊕ (0x03 • 0x{s2:02X}) ⊕ 0x{s3:02X} ───")
    print()
    
    print(f"    Term 1: 0x{s0:02X} (identity)")
    term1 = s0
    print(f"      Value: {term1:08b}b = 0x{term1:02X}")
    print()
    
    print(f"    Term 2: (0x02 • 0x{s1:02X})")
    term2 = gf_mul_verbose(s1, 0x02)
    print()
    
    print(f"    Term 3: (0x03 • 0x{s2:02X})")
    term3 = gf_mul_verbose(s2, 0x03)
    print()
    
    print(f"    Term 4: 0x{s3:02X} (identity)")
    term4 = s3
    print(f"      Value: {term4:08b}b = 0x{term4:02X}")
    print()
    
    print(f"    Final XOR of all terms:")
    print(f"      {term1:08b}b  (0x{term1:02X}) ← Term 1")
    print(f"    ⊕ {term2:08b}b  (0x{term2:02X}) ← Term 2")
    print(f"    ⊕ {term3:08b}b  (0x{term3:02X}) ← Term 3")
    print(f"    ⊕ {term4:08b}b  (0x{term4:02X}) ← Term 4")
    
    r1 = term1 ^ term2 ^ term3 ^ term4
    print(f"    = {r1:08b}b  (0x{r1:02X}) ← r1")
    print()
    
    # Calculate r2 = s0 ⊕ s1 ⊕ (02•s2) ⊕ (03•s3)
    print(f"  ─── Calculating r2 = 0x{s0:02X} ⊕ 0x{s1:02X} ⊕ (0x02 • 0x{s2:02X}) ⊕ (0x03 • 0x{s3:02X}) ───")
    print()
    
    print(f"    Term 1: 0x{s0:02X} (identity)")
    term1 = s0
    print(f"      Value: {term1:08b}b = 0x{term1:02X}")
    print()
    
    print(f"    Term 2: 0x{s1:02X} (identity)")
    term2 = s1
    print(f"      Value: {term2:08b}b = 0x{term2:02X}")
    print()
    
    print(f"    Term 3: (0x02 • 0x{s2:02X})")
    term3 = gf_mul_verbose(s2, 0x02)
    print()
    
    print(f"    Term 4: (0x03 • 0x{s3:02X})")
    term4 = gf_mul_verbose(s3, 0x03)
    print()
    
    print(f"    Final XOR of all terms:")
    print(f"      {term1:08b}b  (0x{term1:02X}) ← Term 1")
    print(f"    ⊕ {term2:08b}b  (0x{term2:02X}) ← Term 2")
    print(f"    ⊕ {term3:08b}b  (0x{term3:02X}) ← Term 3")
    print(f"    ⊕ {term4:08b}b  (0x{term4:02X}) ← Term 4")
    
    r2 = term1 ^ term2 ^ term3 ^ term4
    print(f"    = {r2:08b}b  (0x{r2:02X}) ← r2")
    print()
    
    # Calculate r3 = (03•s0) ⊕ s1 ⊕ s2 ⊕ (02•s3)
    print(f"  ─── Calculating r3 = (0x03 • 0x{s0:02X}) ⊕ 0x{s1:02X} ⊕ 0x{s2:02X} ⊕ (0x02 • 0x{s3:02X}) ───")
    print()
    
    print(f"    Term 1: (0x03 • 0x{s0:02X})")
    term1 = gf_mul_verbose(s0, 0x03)
    print()
    
    print(f"    Term 2: 0x{s1:02X} (identity)")
    term2 = s1
    print(f"      Value: {term2:08b}b = 0x{term2:02X}")
    print()
    
    print(f"    Term 3: 0x{s2:02X} (identity)")
    term3 = s2
    print(f"      Value: {term3:08b}b = 0x{term3:02X}")
    print()
    
    print(f"    Term 4: (0x02 • 0x{s3:02X})")
    term4 = gf_mul_verbose(s3, 0x02)
    print()
    
    print(f"    Final XOR of all terms:")
    print(f"      {term1:08b}b  (0x{term1:02X}) ← Term 1")
    print(f"    ⊕ {term2:08b}b  (0x{term2:02X}) ← Term 2")
    print(f"    ⊕ {term3:08b}b  (0x{term3:02X}) ← Term 3")
    print(f"    ⊕ {term4:08b}b  (0x{term4:02X}) ← Term 4")
    
    r3 = term1 ^ term2 ^ term3 ^ term4
    print(f"    = {r3:08b}b  (0x{r3:02X}) ← r3")
    print()
    
    print(f"  {'='*70}")
    print(f"  Result for column {col_index}: [0x{r0:02X}, 0x{r1:02X}, 0x{r2:02X}, 0x{r3:02X}]")
    print(f"  {'='*70}")
    
    return [r0, r1, r2, r3]


def mix_columns_verbose(state):
    """Apply MixColumns with explicit binary operations."""
    print(f"\n{'='*70}")
    print("MIXCOLUMNS OPERATION (VERBOSE WITH BINARY OPERATIONS)")
    print('='*70)
    print("\nEach column is multiplied by the fixed matrix in GF(2^8):")
    print("┌──────────────┐")
    print("│ 02 03 01 01  │")
    print("│ 01 02 03 01  │")
    print("│ 01 01 02 03  │")
    print("│ 03 01 01 02  │")
    print("└──────────────┘")
    print("\nGF(2^8) multiplication rules:")
    print("  • Multiply by 0x02: Left shift, XOR with 0x1B if high bit was set")
    print("  • Multiply by 0x03: (0x02 • a) ⊕ a")
    print("  • Multiply by 0x01: Identity (no change)")
    print("  • Irreducible polynomial: x^8 + x^4 + x^3 + x + 1 = 0x11B")
    print("  • Reduction modulo: 0x1B (0x11B with high bit implicit)")
    
    out = [[0]*4 for _ in range(4)]
    
    for c in range(4):
        col = [state[r][c] for r in range(4)]
        mixed = mix_single_column_verbose(col, col_index=c)
        for r in range(4):
            out[r][c] = mixed[r]
    
    # Show before/after
    print(f"\n{'INPUT STATE':^22}      {'OUTPUT STATE':^22}")
    print("┌─────────────────┐      ┌─────────────────┐")
    for r in range(4):
        left = '  '.join(f"{state[r][c]:02X}" for c in range(4))
        right = '  '.join(f"{out[r][c]:02X}" for c in range(4))
        print(f"│ {left} │  =>  │ {right} │")
    print("└─────────────────┘      └─────────────────┘")
    
    return out


# Test with a simple example
print("\n" + "="*70)
print("EXAMPLE: MixColumns on a single column [0x63, 0x63, 0x63, 0x63]")
print("="*70)

test_state = [
    [0x63, 0x00, 0x00, 0x00],
    [0x63, 0x00, 0x00, 0x00],
    [0x63, 0x00, 0x00, 0x00],
    [0x63, 0x00, 0x00, 0x00]
]

#result = mix_columns_verbose(test_state)


EXAMPLE: MixColumns on a single column [0x63, 0x63, 0x63, 0x63]


## AES keys expansion





In [2]:
# AES Key Expansion (Key schedule) — RotWord, SubWord, Rcon, key_expansion

def rot_word(word):
    """Rotate a word (4-byte list) left by one byte."""
    result = [word[1], word[2], word[3], word[0]]
    return result


def sub_word(word, sbox):
    """Apply S-box to each byte of the 4-byte word."""
    result = [sbox[b] for b in word]
    return result


def compute_rcon(n):
    """Compute Rcon list up to index n (1-based). Rcon[1] = 0x01.
    Returns list where rcon[i] is the byte for iteration i (i starting at 1).
    """
    rcon = [0]* (n+1)
    rcon[1] = 0x01
    for i in range(2, n+1):
        rcon[i] = gf_mul(rcon[i-1], 0x02)
    return rcon


def key_expansion_verbose(key_bytes):
    """Expand a cipher key (bytes) into the AES key schedule with verbose output.
    Accepts key_bytes as an iterable of 16/24/32 bytes (AES-128/192/256).
    Returns list of 4-byte words (each a list of 4 ints) forming the expanded key.
    """
    if isinstance(key_bytes, bytes):
        key = list(key_bytes)
    else:
        key = list(key_bytes)

    Nk = len(key) // 4  # number of 32-bit words in key
    if Nk not in (4, 6, 8):
        raise ValueError('Key must be 16, 24 or 32 bytes long')

    # Number of rounds
    Nr = {4:10, 6:12, 8:14}[Nk]
    Nb = 4
    n_words = Nb * (Nr + 1)

    print(f"\n{'='*80}")
    print(f"AES KEY EXPANSION - AES-{Nk*32}")
    print(f"{'='*80}")
    print(f"Key length: {len(key)} bytes ({Nk} words)")
    print(f"Number of rounds: {Nr}")
    print(f"Total words needed: {n_words}")
    print(f"Original key: {' '.join(f'{b:02X}' for b in key)}")
    print(f"{'='*80}\n")

    # Fill initial words (first Nk words) from the key (word = 4 bytes)
    w = []
    print("INITIAL KEY WORDS (from original key):")
    for i in range(Nk):
        word = key[4*i:4*i+4]
        w.append(word)
        print(f"  W[{i:2d}] = [{' '.join(f'{b:02X}' for b in word)}]")
    print()

    # Compute Rcon up to needed count
    rcon = compute_rcon((n_words // Nk) + 1)
    print("RCON VALUES:")
    for i in range(1, min(11, len(rcon))):
        print(f"  Rcon[{i}] = 0x{rcon[i]:02X}")
    print()

    # Get sbox
    sbox = sbox_generated if 'sbox_generated' in globals() else (AES_SBOX if 'AES_SBOX' in globals() else sbox_ref)

    # Expand
    print("KEY EXPANSION PROCESS:")
    print("="*80)
    for i in range(Nk, n_words):
        print(f"\nGenerating W[{i:2d}]:")
        print(f"  {'─'*76}")
        
        temp = w[i-1].copy()
        print(f"  Start with W[{i-1:2d}] = [{' '.join(f'{b:02X}' for b in temp)}]")
        
        # Check if we need special transformations
        if i % Nk == 0:
            # This is a special word (multiple of Nk)
            round_num = i // Nk
            print(f"  ✓ i % Nk == 0 (i={i}, Nk={Nk}) → Apply RotWord, SubWord, and Rcon[{round_num}]")
            
            # RotWord
            rotated = rot_word(temp)
            print(f"    1. RotWord:  [{' '.join(f'{b:02X}' for b in temp)}] → [{' '.join(f'{b:02X}' for b in rotated)}]")
            
            # SubWord
            substituted = sub_word(rotated, sbox)
            print(f"    2. SubWord:  [{' '.join(f'{b:02X}' for b in rotated)}] → [{' '.join(f'{b:02X}' for b in substituted)}]")
            
            # XOR with Rcon
            r = rcon[round_num]
            temp = substituted.copy()
            temp[0] ^= r
            print(f"    3. XOR Rcon: [{' '.join(f'{b:02X}' for b in substituted)}] ⊕ [0x{r:02X}, 00, 00, 00]")
            print(f"       Result:   [{' '.join(f'{b:02X}' for b in temp)}]")
            
        elif Nk > 6 and i % Nk == 4:
            # Extra SubWord for AES-256
            print(f"  ✓ AES-256 and i % Nk == 4 (i={i}, Nk={Nk}) → Apply SubWord only")
            substituted = sub_word(temp, sbox)
            print(f"    SubWord: [{' '.join(f'{b:02X}' for b in temp)}] → [{' '.join(f'{b:02X}' for b in substituted)}]")
            temp = substituted
        else:
            print(f"  ✗ No special transformation (i % Nk = {i % Nk})")
        
        # w[i] = w[i-Nk] XOR temp
        prev = w[i-Nk]
        new_word = [prev[j] ^ temp[j] for j in range(4)]
        
        print(f"\n  Final XOR with W[{i-Nk:2d}]:")
        print(f"    W[{i-Nk:2d}]  = [{' '.join(f'{b:02X}' for b in prev)}]")
        print(f"    temp    = [{' '.join(f'{b:02X}' for b in temp)}]")
        print(f"    ─────────────────────────────")
        print(f"    W[{i:2d}]  = [{' '.join(f'{b:02X}' for b in new_word)}]")
        
        w.append(new_word)
        
        # Show round key completion
        if (i + 1) % 4 == 0:
            round_num = (i + 1) // 4 - 1
            print(f"\n  ✓✓✓ Round {round_num} key complete (W[{i-3}] through W[{i}]) ✓✓✓")

    print(f"\n{'='*80}")
    print("KEY EXPANSION COMPLETE")
    print(f"{'='*80}\n")
    
    return w


def key_expansion(key_bytes):
    """Non-verbose version for regular use."""
    if isinstance(key_bytes, bytes):
        key = list(key_bytes)
    else:
        key = list(key_bytes)

    Nk = len(key) // 4
    if Nk not in (4, 6, 8):
        raise ValueError('Key must be 16, 24 or 32 bytes long')

    Nr = {4:10, 6:12, 8:14}[Nk]
    Nb = 4
    n_words = Nb * (Nr + 1)

    w = []
    for i in range(Nk):
        word = key[4*i:4*i+4]
        w.append(word)

    rcon = compute_rcon((n_words // Nk) + 1)
    sbox = sbox_generated if 'sbox_generated' in globals() else (AES_SBOX if 'AES_SBOX' in globals() else sbox_ref)

    for i in range(Nk, n_words):
        temp = w[i-1].copy()
        if i % Nk == 0:
            temp = sub_word(rot_word(temp), sbox)
            r = rcon[i//Nk]
            temp[0] ^= r
        elif Nk > 6 and i % Nk == 4:
            temp = sub_word(temp, sbox)
        prev = w[i-Nk]
        new_word = [prev[j] ^ temp[j] for j in range(4)]
        w.append(new_word)

    return w


def words_to_round_key_matrix(words, round_index):
    """Produce a 4x4 round key matrix (rows x cols) for given round_index."""
    start = round_index * 4
    round_words = words[start:start+4]
    matrix = [[round_words[col][row] for col in range(4)] for row in range(4)]
    return matrix


# Demo: expand the AES test key from FIPS-197
#sample_key_hex = '2b7e151628aed2a6abf7158809cf4f3c'
#                 
#sample_key = bytes.fromhex(sample_key_hex)
#
#print("\n" + "="*80)
#print("VERBOSE KEY EXPANSION DEMO")
#print("="*80)
#
#expanded = key_expansion_verbose(sample_key)
#
## Print round keys summary
#Nk = len(sample_key)//4
#Nr = {4:10, 6:12, 8:14}[Nk]
#
#print("\n" + "="*80)
#print("ROUND KEYS SUMMARY")
#print("="*80)
#for r in range(Nr+1):
#    mat = words_to_round_key_matrix(expanded, r)
#    flat = [mat[row][col] for col in range(4) for row in range(4)]
#    print(f'Round {r:02d} key: {" ".join(f"{b:02X}" for b in flat)}')

# key bytes 0x24 0x75 0xA2 0xB3 0x34 0x75 0x56 0x88 0x31 0xe2 0x12 0x00 0x13 0xaa 0x54 0x87
#sample_key_hex = '2475a2b33475568831e2120013aa5487'
#sample_key = bytes.fromhex(sample_key_hex)  # <-- ADD THIS LINE
#key_expansion_verbose(sample_key)

In [8]:
print('\nAES FUll')

def print_state(state, title=None):
    if title:
        print(title)
    for r in range(4):
        print(' '.join(f"{state[r][c]:02X}" for c in range(4)))
    print()

def AES_encrypt(plaintext, key):
    
    # From string to hex bytes
    plaintext = plaintext.encode('utf-8')
    key = key.encode('utf-8')

    # Pad plaintext and key to 16 bytes if needed
    plaintext = plaintext.ljust(16, b'\x00')[:16]
    key = key.ljust(16, b'\x00')[:16]
    
    # write the plaintext and the key in matrix form
    state = [[plaintext[r + 4*c] for c in range(4)] for r in range(4)]
    print_state(state, 'PlainText in hex:')

    key_state = [[key[r + 4*c] for c in range(4)] for r in range(4)]
    print_state(key_state, 'Key in hex:')

    # Key Expansion
    expanded_key = key_expansion_verbose(key)
    round_keys = [words_to_round_key_matrix(expanded_key, r) for r in range(11)]

    print("\n" + "="*80)
    print("AES ENCRYPTION PROCESS")
    print("="*80 + "\n")
    
    # Print round keys
    for r in range(11):
        print_state(round_keys[r], f'Round Key {r}:')
    
    # Initial AddRoundKey
    state = add_round_key(state, round_keys[0])
    print_state(state, 'After Initial AddRoundKey:')
    
    # 9 Main Rounds
    for round in range(1, 10):
        state = sub_bytes(state, AES_SBOX)
        state = shift_rows(state)
        state = mix_columns_verbose(state)
        state = add_round_key(state, round_keys[round])
        print_state(state, f'After Round {round}:')

    # Final Round (no MixColumns)
    state = sub_bytes(state, AES_SBOX)
    state = shift_rows(state)
    state = add_round_key(state, round_keys[10])
    print_state(state, 'After Final Round:')

    # Serialize state to ciphertext
    ciphertext = bytearray(16)
    for r in range(4):
        for c in range(4):
            ciphertext[r + 4*c] = state[r][c]


    return ciphertext
    




AES FUll


In [9]:
def AES_decrypt(ciphertext, key):    
    # From string to hex bytes
    key = key.encode('utf-8')
    # Pad ciphertext and key to 16 bytes if needed
    ciphertext = ciphertext.ljust(16, b'\x00')[:16]
    key = key.ljust(16, b'\x00')[:16]
    # write the ciphertext and the key in matrix form
    state = [[ciphertext[r + 4*c] for c in range(4)] for r in range(4)]
    print_state(state, 'CipherText in hex:')
    key_state = [[key[r + 4*c] for c in range(4)] for r in range(4)]
    print_state(key_state, 'Key in hex:')
    # Key Expansion
    expanded_key = key_expansion(key)
    round_keys = [words_to_round_key_matrix(expanded_key, r) for r in range(11)]
    print("\n" + "="*80)
    print("AES DECRYPTION PROCESS")
    print("="*80 + "\n")
   
    # Initial AddRoundKey
    state = add_round_key(state, round_keys[10])
    print_state(state, 'After Initial AddRoundKey:')
    # 9 Main Rounds
    for round in range(9, 0, -1):
        state = inv_shift_rows(state)
        state = inv_sub_bytes(state, inv_sbox_ref)
        state = add_round_key(state, round_keys[round])
        state = inv_mix_columns(state)
        print_state(state, f'After Round {round}:')

    # Final Round (no InvMixColumns)
    state = inv_shift_rows(state)
    state = inv_sub_bytes(state, inv_sbox_ref)
    state = add_round_key(state, round_keys[0])
    print_state(state, 'After Final Round:')
    # Serialize state to plaintext
    plaintext = bytearray(16)

    for r in range(4):
        for c in range(4):
            plaintext[r + 4*c] = state[r][c]

    print('\nAES Decrypted PlainText in hex:')
    print(' '.join(f"{b:02X}" for b in plaintext))
    # From hex bytes to string
    plaintext = plaintext.rstrip(b'\x00').decode('utf-8')
    print('\nAES Decrypted PlainText:')
    print(plaintext)
    return plaintext





In [ ]:
    
# Implementation
#C = AES_encrypt("MEETME", "MEETME")

#D = AES_decrypt(C, "MEETME")